# Crypto Momentum / Trend

This notebook includes analysis on momentum effects in Cryptocurrencies. We analyze both cross-sectional and time-series momentum, and look for possibile methods of monetizing these effects.

Our dataset comes from trading history for pairs on the Kraken exchange ([link](https://support.kraken.com/hc/en-us/articles/360047543791-Downloadable-historical-market-data-time-and-sales-)). We further constrain our analysis to USD pairs only. Kraken is our primary trading exchange due to geographical restrictions (US...zzz) and Kraken's superior fee structure as compared to other US exchanges like Coinbase.

## Load Data

In [ ]:
import argparse
import pandas as pd
import plotly.express as px
from scipy import stats
import numpy as np
from datetime import datetime
import pytz
from pathlib import Path
from numba import njit, jit
import vectorbt as vbt

from analysis.analysis import analysis

from simulation.simulation import simulation
from simulation.backtest import backtest_crypto
from simulation.optimize import optimize_crypto
from signal_generation.signal_generation import (
    create_analysis_signals,
    create_trading_signals,
)
from signal_generation.constants import SignalType
from signal_generation.rohrbach import create_rohrbach_signals
from signal_generation.common import (
    ema,
    volatility_ema,
    bins,
    volatility,
    future_volatility,
)
from position_generation.benchmark import generate_benchmark_btc
from position_generation.constants import (
    NUM_LONG_ASSETS_COL,
    NUM_SHORT_ASSETS_COL,
    NUM_KEPT_ASSETS_COL,
    SCALED_POSITION_COL,
    NUM_OPEN_LONG_POSITIONS_COL,
    NUM_OPEN_SHORT_POSITIONS_COL,
    NUM_OPEN_POSITIONS_COL,
)
from position_generation.utils import nonempty_positions
from position_generation.generate_positions import generate_positions
from position_generation.utils import Direction
from data.utils import load_ohlc_to_hourly_filtered, load_ohlc_to_daily_filtered
from core.utils import filter_universe
from core.constants import (
    in_universe_excl_stablecoins,
    in_shitcoin_trending_universe,
    in_mature_trending_universe,
)

np.set_printoptions(linewidth=1000)
pd.set_option("display.width", 2000)
pd.set_option("display.precision", 3)
pd.set_option("display.float_format", "{:.3f}".format)

In [ ]:
input_path = "/home/elo/data/usd_ohlc_fixed.csv"
input_freq = "1h"
start_date = "2020/01/01"
end_date = "2023/12/31"
tz = pytz.timezone("UTC")
start_date = tz.localize(datetime.strptime(start_date.replace("/", "-"), "%Y-%m-%d"))
end_date = tz.localize(datetime.strptime(end_date.replace("/", "-"), "%Y-%m-%d"))

# Parse data
df_daily = load_ohlc_to_daily_filtered(
    input_path, input_freq=input_freq, tz=tz, whitelist_fn=in_universe_excl_stablecoins
)

# Create signals
df_analysis = create_analysis_signals(df_daily, periods_per_day=1)

# Validate dates
data_start = df_analysis["timestamp"].min()
if start_date < data_start:
    print(f"Input start_date is before start of data! Setting to {data_start}")
    start_date = data_start
data_end = df_analysis["timestamp"].max()
if end_date > data_end:
    print(f"Input end_date is after end of data! Setting to {data_end}")
    end_date = data_end

## Data Analysis

### Introductory Analysis - Past Returns vs Future Returns

Let's try the simplest, dumbest thing first. Look for a relationship between past (30d) returns and future (14-28d) returns.

In [ ]:
analysis(
    df_analysis,
    feature="30d_log_returns",
    target="next_7d_log_returns",
    bin_feature="30d_log_returns_decile",
)

In [ ]:
analysis(
    df_analysis,
    feature="30d_log_returns",
    target="next_14d_log_returns",
    bin_feature="30d_log_returns_decile",
)

In [ ]:
analysis(
    df_analysis,
    feature="30d_log_returns",
    target="next_28d_log_returns",
    bin_feature="30d_log_returns_decile",
)

We see evidence of momentum effects using 30 day past returns. The explanatory power of 30d returns seems to increase as the future lookahead horizon increases (28d > 14d > 7d).

The effect seems to be quite noisy year to year. Is there a relationship between momentum and whether the market was going up/down? Let's use BTC as a proxy for the market and look at returns per year.

In [ ]:
# Get years where BTC returns were positive
df_tmp = (
    df_analysis.loc[df_analysis["ticker"] == "BTC/USD"]
    .groupby(["year", "ticker"])
    .agg(
        {
            "returns": "sum",
        }
    )
    .reset_index(0)
)
df_tmp["up_year"] = df_tmp["returns"] > 0
df_tmp

In [ ]:
# Analyze momentum effects for up years
up_years = [2013, 2015, 2016, 2017, 2019, 2020, 2021, 2023]

feature = "30d_log_returns"
bin_feature = "30d_log_returns_decile"
target = "next_28d_log_returns"

# Plot de-meaned future returns over 30d return deciles per year
for year in up_years:
    df_tmp = (
        df_analysis.loc[df_analysis["year"] == year]
        .groupby(
            [
                bin_feature,
            ]
        )
        .agg({target: "mean"})
        .reset_index()
    )
    # De-mean
    df_tmp[target] = df_tmp[target] - df_tmp[target].mean()
    # Plot
    fig = px.bar(
        df_tmp,
        x=bin_feature,
        y=target,
        title=year,
    )
    fig.show()

There are still some exceptions (2015, 2020), but overall the effect seems to persist. The exceptional years support the notion that harnessing this effect is somewhat shitty, and so is likely to persist.

### Effects of Volume on Relationship

Our hypothesis for why momentum exists includes both behavioral reasons (FOMO, flows yolo-ing into coins going up) as well as limits to arbitrage (kinda risky/shitty to take the other side of such a volatile trade). If this is true, we would expect to see the relationship strengthen for the really shitty shitcoins (where limits to arbitrage are greater due to limited capacity).

In [ ]:
from scipy import stats

df_effect_vs_volume = pd.DataFrame(
    {
        "max_dollar_volume": pd.Series(dtype="int"),
        "slope": pd.Series(dtype="float"),
        "r2": pd.Series(dtype="float"),
        "num_data_points": pd.Series(dtype="int"),
    }
)
feature = "30d_log_returns"
target = "next_28d_log_returns"
bin_feature = "30d_log_returns_decile"
for dollar_volume in [
    np.inf,
    100e6,
    10e6,
    1e6,
    500000,
    100000,
    50000,
    40000,
    30000,
    20000,
    10000,
    5000,
    2000,
    1000,
    100,
]:
    volume_mask = (df_analysis["dollar_volume"] <= dollar_volume) & (
        df_analysis["dollar_volume"] > 0
    )
    df_tmp = df_analysis.loc[volume_mask].dropna()
    # Linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        df_tmp[feature], df_tmp[target]
    )
    num_data_points = len(df_tmp)
    df_effect_vs_volume.loc[len(df_effect_vs_volume.index)] = [
        dollar_volume,
        slope,
        r_value**2,
        num_data_points,
    ]

df_effect_vs_volume

The data seems to support this hypothesis up to a max daily volume of ~\$10000.

A follow-up question: is this driven by some shitcoin outliers? Let's plot the scatterplot for the low dollar volume datapoints to see.

In [ ]:
feature = "30d_log_returns"
target = "next_28d_log_returns"
bin_feature = "30d_log_returns_decile"
volume_mask = (df_analysis["dollar_volume"] <= 20000) & (
    df_analysis["dollar_volume"] > 0
)
analysis(
    df_analysis.loc[volume_mask],
    feature=feature,
    target=target,
    bin_feature=bin_feature,
)

In [ ]:
# Print outliers
df_analysis.dropna().loc[volume_mask].sort_values(
    by="next_28d_log_returns", ascending=False
)[
    [
        "ticker",
        "timestamp",
        "volume",
        "dollar_volume",
        "30d_returns",
        "next_28d_log_returns",
    ]
].head(
    50
)

In [ ]:
# Filter outliers from analysis
feature = "30d_log_returns"
target = "next_28d_log_returns"
bin_feature = "30d_log_returns_decile"
analysis(
    df_analysis.loc[volume_mask & (df_analysis["next_28d_log_returns"] <= 2.0)],
    feature=feature,
    target=target,
    bin_feature=bin_feature,
)

Even after we filter out the egregiously high returns (> 100% over the next month), the relationship seems to hold up.

At this point, we can be reasonably confident of the following:
- Momentum effects exist in cryptocurrency markets
- Effects seem to strengthen inversely proportional to daily traded volume

### Trend Overextension

There's a phenomenon known as "trend overextension" which describes the fact that very large signals of trend may actually predict reversion in future returns (due to the trend being overextended, more capital is willing to take the other side of the trade to bring prices back down to "fair value").

Do we see this in our cryptocurrency data?

In [ ]:
# Create centiles
bin_feature = "30d_log_returns_centile"
df_analysis[bin_feature] = bins(df_analysis, column="30d_log_returns", num_bins=100)
# Create rank
rank_feature = "30d_log_returns_rank"
df_analysis[rank_feature] = (
    df_analysis.groupby(["timestamp"])["30d_log_returns"]
    .fillna(value=0)
    .rank(method="dense", ascending=False)
    .astype(int)
)
# Remove outliers
df_analysis_filtered = df_analysis.loc[df_analysis["next_28d_log_returns"] < 2.0]


feature = rank_feature
target = "next_28d_log_returns"

# All Data
df_tmp = (
    df_analysis_filtered.groupby([rank_feature]).agg({target: "mean"}).reset_index()
)
fig = px.bar(df_tmp, x=rank_feature, y=target, title="All Data")
fig.show()

# # Low Volume
# df_tmp = (
#     df_analysis_filtered.loc[df_analysis["dollar_volume"] <= 10000]
#     .groupby([rank_feature])
#     .agg({target: "mean"})
#     .reset_index()
# )
# fig = px.bar(df_tmp, x=rank_feature, y=target, title="Dollar Volume <= $10,000")
# fig.show()

# # High Volume
# df_tmp = (
#     df_analysis_filtered.loc[df_analysis["dollar_volume"] >= 1000000]
#     .groupby([rank_feature])
#     .agg({target: "mean"})
#     .reset_index()
# )
# fig = px.bar(df_tmp, x=rank_feature, y=target, title="Dollar Volume >= $1,000,000")
# fig.show()

I can't really say I see evidence of overextension to be honest...I thought I had seen it in a previous analysis but I can't really reproduce. I mean, there's maybe some evidence beyond the top ~10% but then it shoots back up in the last 4%.

There's maybe some evidence for it in the low volume tickers.

This is relevant for deciding which activation function to use (sigmoid vs $x * exp(-x^2)$).

## Trend Signal (Rohrbach et. al 2017)

Rohrbach and coauthors published a 2017 paper titled "Momentum and trend following trading strategies for currencies and bitcoin" ([link](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2949379)) in which they describe a formula for a trend signal. Let's take a look at how well this predicts returns.

In [ ]:
# The Rohrbach signal is already generated under "trend_signal"
df_analysis = create_rohrbach_signals(df_analysis, periods_per_day=1)

feature = "trend_signal"
target = "next_28d_log_returns"
bin_feature = "trend_decile"
analysis(
    df_analysis,
    feature=feature,
    target=target,
    bin_feature=bin_feature,
)

The shape of the decile (cross-sectional) bar plots look roughly the same as the plots using 30d returns, which is a good sign. In the scatterplot, the r_value is only marginally higher but the plot's outliers look a lot better (more up and to the right).

How about the volume-filtered analysis?

In [ ]:
df_trend_vs_volume = pd.DataFrame(
    {
        "max_dollar_volume": pd.Series(dtype="int"),
        "slope": pd.Series(dtype="float"),
        "r2": pd.Series(dtype="float"),
        "num_data_points": pd.Series(dtype="int"),
    }
)
feature = "trend_signal"
target = "next_28d_log_returns"
bin_feature = "trend_decile"
for dollar_volume in [
    np.inf,
    100e6,
    10e6,
    1e6,
    500000,
    100000,
    50000,
    40000,
    30000,
    20000,
    10000,
    5000,
    2000,
    1000,
    100,
]:
    volume_mask = (df_analysis["dollar_volume"] <= dollar_volume) & (
        df_analysis["dollar_volume"] > 0
    )
    df_tmp = df_analysis.loc[volume_mask].dropna()
    # Linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        df_tmp[feature], df_tmp[target]
    )
    num_data_points = len(df_tmp)
    df_trend_vs_volume.loc[len(df_trend_vs_volume.index)] = [
        dollar_volume,
        slope,
        r_value**2,
        num_data_points,
    ]

df_trend_vs_volume

In [ ]:
feature = "trend_signal"
target = "next_28d_log_returns"
bin_feature = "trend_decile"
volume_mask = (df_analysis["dollar_volume"] <= 100000) & (
    df_analysis["dollar_volume"] > 0
)
analysis(
    df_analysis.loc[volume_mask],
    feature=feature,
    target=target,
    bin_feature=bin_feature,
)

In [ ]:
# Filter outliers from analysis
feature = "trend_signal"
target = "next_28d_log_returns"
bin_feature = "trend_decile"
analysis(
    df_analysis.loc[volume_mask & (df_analysis["next_28d_log_returns"] <= 2.0)],
    feature=feature,
    target=target,
    bin_feature=bin_feature,
)

As before, effects are stronger as volume decreases (although with this signal, caps out around \$1 - \$10k daily volume). Effects persist even after removing egregious outliers.

What happens with the higher volume data?

In [ ]:
# Look at higher volume data only
feature = "trend_signal"
target = "next_28d_log_returns"
bin_feature = "trend_decile"
volume_mask = (df_analysis["dollar_volume"] >= 1e6) & (df_analysis["dollar_volume"] > 0)
analysis(
    df_analysis.loc[volume_mask],
    feature=feature,
    target=target,
    bin_feature=bin_feature,
)

Despite the r_value being smaller, we see this hugely obvious positive linear relationship at the extremes. What are these data? Is this a special case/one-off?

In [ ]:
df_analysis.dropna().loc[volume_mask].sort_values(
    by="next_28d_log_returns", ascending=False
)[
    ["ticker", "timestamp", "dollar_volume", "trend_signal", "next_28d_log_returns"]
].head(
    50
)

It's the great DOGE pump of 2021 as well as a pump in 2017 (unaware). What happens if we remove these outliers?

In [ ]:
# Filter outliers from analysis
feature = "trend_signal"
target = "next_28d_log_returns"
bin_feature = "trend_decile"
analysis(
    df_analysis.loc[volume_mask & (df_analysis["next_28d_log_returns"] <= 2.0)],
    feature=feature,
    target=target,
    bin_feature=bin_feature,
)

We still see a generally positive relationship between trend and future returns even with the outliers removed.

At this point, we can probably state the following:
- Trend effects are stronger on average in lower volume tickers (up to a certain point).
- High volume tickers contain some of the most egregiously high trend <-> returns data points. This may align with our hypothesis for where trends come from: in order to see an egregiously strong trend, you need investors to ape into the same coin (which will drive up volumes).
- We want to target the low volume universe due to stronger trend effects on average, but also want to include the high volume universe to capture any future manias (also trend is still generally present in high volume).

### Sigmoid Activation Function

In [ ]:
df_analysis["trend_sigmoid_decile"] = bins(
    df_analysis, column="trend_signal_sigmoid", num_bins=10
)

# The Rohrbach signal w/ sigmoid activation is already generated under "trend_signal_sigmoid"
feature = "trend_signal_sigmoid"
target = "next_28d_log_returns"
bin_feature = "trend_sigmoid_decile"
analysis(
    df_analysis,
    feature=feature,
    target=target,
    bin_feature=bin_feature,
)

Sigmoid seems marginally better than exponential, at least in data analysis. Correlation is roughly the same (as expected, both are just transformations of the same signal), but slope is ~2x higher.

Does the answer change as a function of volume?

In [ ]:
df_trend_vs_volume = pd.DataFrame(
    {
        "max_dollar_volume": pd.Series(dtype="int"),
        "slope_exponential": pd.Series(dtype="float"),
        "r2_exponential": pd.Series(dtype="float"),
        "slope_sigmoid": pd.Series(dtype="float"),
        "r2_sigmoid": pd.Series(dtype="float"),
        "num_data_points": pd.Series(dtype="int"),
    }
)
feature_exponential = "trend_signal"
feature_sigmoid = "trend_signal_sigmoid"
target = "next_28d_log_returns"
for dollar_volume in [
    np.inf,
    100e6,
    10e6,
    1e6,
    500000,
    100000,
    50000,
    40000,
    30000,
    20000,
    10000,
    5000,
    2000,
    1000,
    100,
]:
    volume_mask = (df_analysis["dollar_volume"] <= dollar_volume) & (
        df_analysis["dollar_volume"] > 0
    )
    df_tmp = df_analysis.loc[volume_mask].dropna()
    # Linear regression
    (
        slope_exponential,
        intercept,
        r_value_exponential,
        p_value,
        std_err,
    ) = stats.linregress(df_tmp[feature_exponential], df_tmp[target])
    slope_sigmoid, intercept, r_value_sigmoid, p_value, std_err = stats.linregress(
        df_tmp[feature_sigmoid], df_tmp[target]
    )
    num_data_points = len(df_tmp)
    df_trend_vs_volume.loc[len(df_trend_vs_volume.index)] = [
        dollar_volume,
        slope_exponential,
        r_value_exponential**2,
        slope_sigmoid,
        r_value_sigmoid**2,
        num_data_points,
    ]

df_trend_vs_volume

In [ ]:
# The Rohrbach signal w/ sigmoid activation is already generated under "trend_signal_sigmoid"
target = "next_28d_log_returns"
bin_feature_exponential = "trend_decile"
bin_feature_sigmoid = "trend_sigmoid_decile"

# All Data
df_tmp = (
    df_analysis.groupby([bin_feature_exponential]).agg({target: "mean"}).reset_index()
)
fig = px.bar(
    df_tmp, x=bin_feature_exponential, y=target, title="All Data - Exponential"
)
fig.show()
df_tmp = df_analysis.groupby([bin_feature_sigmoid]).agg({target: "mean"}).reset_index()
fig = px.bar(df_tmp, x=bin_feature_sigmoid, y=target, title="All Data - Sigmoid")
fig.show()

# Low Volume
df_tmp = (
    df_analysis.loc[
        (df_analysis["dollar_volume"] <= 100000) & (df_analysis["dollar_volume"] > 0)
    ]
    .groupby([bin_feature_exponential])
    .agg({target: "mean"})
    .reset_index()
)
fig = px.bar(
    df_tmp,
    x=bin_feature_exponential,
    y=target,
    title="Dollar Volume <= $100,000 - Exponential",
)
fig.show()
df_tmp = (
    df_analysis.loc[
        (df_analysis["dollar_volume"] <= 100000) & (df_analysis["dollar_volume"] > 0)
    ]
    .groupby([bin_feature_sigmoid])
    .agg({target: "mean"})
    .reset_index()
)
fig = px.bar(
    df_tmp, x=bin_feature_sigmoid, y=target, title="Dollar Volume <= $100,000 - Sigmoid"
)
fig.show()

# High Volume
df_tmp = (
    df_analysis.loc[df_analysis["dollar_volume"] >= 1000000]
    .groupby([bin_feature_exponential])
    .agg({target: "mean"})
    .reset_index()
)
fig = px.bar(
    df_tmp,
    x=bin_feature_exponential,
    y=target,
    title="Dollar Volume >= $1,000,000 - Exponential",
)
fig.show()
df_tmp = (
    df_analysis.loc[df_analysis["dollar_volume"] >= 1000000]
    .groupby([bin_feature_sigmoid])
    .agg({target: "mean"})
    .reset_index()
)
fig = px.bar(
    df_tmp,
    x=bin_feature_sigmoid,
    y=target,
    title="Dollar Volume >= $1,000,000 - Sigmoid",
)
fig.show()

Sigmoid just seems better at every level of volume.

**Takeaways:**
- **Use sigmoid activation function**
- **If incorporating cross-sectional momentum,**
  - **For low volume universe keep top 3 deciles (top 30%)**
  - **For high volume universe keep top 2 deciles (top 15-20%)**

## Combined Model - Multivariate OLS

What other factors can we combine with trend to predict returns? Some ideas I've seen mentioned in other places include: funding rates, basis, borrowing rates.

In [ ]:
import statsmodels.formula.api as sm

# Rename column starting with number
df_analysis["thirty_day_returns"] = df_analysis["30d_returns"]
df_analysis["thirty_day_dollar_volume"] = df_analysis["30d_dollar_volume"]

features = ["thirty_day_returns", "thirty_day_dollar_volume"]
target = "next_28d_returns"
result = sm.ols(
    formula=f"{target} ~ {' + '.join(features)}", data=df_analysis.dropna()
).fit()
print(result.summary())

print(result.rsquared)

# Workspace

### Trend Signal Rolling Absolute Averages

In [ ]:
print(f'Trend Signal Mean: {df_analysis["trend_signal"].dropna().mean():.2f}')
print(
    f'Positive Trend Signal Mean: {df_analysis.loc[df_analysis["trend_signal"] >= 0]["trend_signal"].dropna().mean():.2f}'
)
print(
    f'Negative Trend Signal Mean: {df_analysis.loc[df_analysis["trend_signal"] < 0]["trend_signal"].dropna().mean():.2f}'
)
print(
    f'Absolute Trend Signal Mean: {np.abs(df_analysis["trend_signal"].dropna()).mean():.2f}'
)
print(f'Trend Signal Median: {df_analysis["trend_signal"].dropna().median():.2f}')
fig = px.histogram(df_analysis, x="trend_signal")
fig.show()

df_analysis["abs_trend_signal"] = np.abs(df_analysis["trend_signal"])
df_tmp = (
    df_analysis.groupby(["timestamp"]).agg({"abs_trend_signal": "mean"}).reset_index()
)
df_tmp["abs_trend_signal_30d_ema"] = (
    df_tmp["abs_trend_signal"].ewm(span=180, adjust=True, ignore_na=False).mean()
)
fig = px.line(df_tmp, x="timestamp", y="abs_trend_signal_30d_ema")
fig.show()

df_positive_only = df_analysis.loc[df_analysis["trend_signal"] > 0]
df_tmp = (
    df_positive_only.groupby("timestamp").agg({"trend_signal": "mean"}).reset_index()
)
df_tmp["pos_trend_signal_30d_ema"] = (
    df_tmp["trend_signal"].ewm(span=180, adjust=True, ignore_na=False).mean()
)
fig = px.line(df_tmp, x="timestamp", y="pos_trend_signal_30d_ema")
fig.show()

In [ ]:
print(f'Trend Signal Mean: {df_analysis["trend_signal_sigmoid"].dropna().mean():.2f}')
print(
    f'Positive Trend Signal Mean: {df_analysis.loc[df_analysis["trend_signal_sigmoid"] >= 0]["trend_signal_sigmoid"].mean():.2f}'
)
print(
    f'Negative Trend Signal Mean: {df_analysis.loc[df_analysis["trend_signal_sigmoid"] < 0]["trend_signal_sigmoid"].mean():.2f}'
)
print(
    f'Absolute Trend Signal Mean: {np.abs(df_analysis["trend_signal_sigmoid"].dropna()).mean():.2f}'
)
print(
    f'Trend Signal Median: {df_analysis["trend_signal_sigmoid"].dropna().median():.2f}'
)
fig = px.histogram(df_analysis, x="trend_signal_sigmoid")
fig.show()

df_analysis["abs_trend_signal_sigmoid"] = np.abs(df_analysis["trend_signal_sigmoid"])
df_tmp = (
    df_analysis.groupby(["timestamp"])
    .agg({"abs_trend_signal_sigmoid": "mean"})
    .reset_index()
)
df_tmp["abs_trend_signal_sigmoid_30d_ema"] = (
    df_tmp["abs_trend_signal_sigmoid"]
    .ewm(span=180, adjust=True, ignore_na=False)
    .mean()
)
fig = px.line(df_tmp, x="timestamp", y="abs_trend_signal_sigmoid_30d_ema")
fig.show()

df_positive_only = df_analysis.loc[df_analysis["trend_signal_sigmoid"] > 0]
df_tmp = (
    df_positive_only.groupby("timestamp")
    .agg({"trend_signal_sigmoid": "mean"})
    .reset_index()
)
df_tmp["pos_trend_signal_sigmoid_30d_ema"] = (
    df_tmp["trend_signal_sigmoid"].ewm(span=180, adjust=True, ignore_na=False).mean()
)
fig = px.line(df_tmp, x="timestamp", y="pos_trend_signal_sigmoid_30d_ema")
fig.show()

### Number of Open Positions

In [ ]:
signal = "trend_signal"
periods_per_day = 1
direction = Direction.LongOnly
vol_target = 0.35
cross_sectional_percentage = 0.2
min_daily_volume = None
max_daily_volume = None

df_analysis = create_trading_signals(
    df_daily, periods_per_day=periods_per_day, signal_type=SignalType.Rohrbach
)
df_positions = generate_positions(
    df_analysis,
    signal=signal,
    periods_per_day=periods_per_day,
    direction=direction,
    vol_target=vol_target,
    cross_sectional_percentage=cross_sectional_percentage,
    min_daily_volume=min_daily_volume,
    max_daily_volume=max_daily_volume,
)

In [ ]:
valid_positions_mask = ~df_positions["scaled_position"].isna()
print(df_positions.loc[valid_positions_mask]["scaled_position"].max())

df_tmp = (
    df_positions.groupby(["timestamp"])
    .agg(
        {
            "scaled_position": "sum",
            NUM_OPEN_LONG_POSITIONS_COL: "first",
            NUM_OPEN_SHORT_POSITIONS_COL: "first",
            NUM_OPEN_POSITIONS_COL: "first",
        }
    )
    .reset_index()
)
fig = px.line(df_tmp, x="timestamp", y=["scaled_position"])
fig.show()
fig = px.line(
    df_tmp,
    x="timestamp",
    y=[
        NUM_OPEN_LONG_POSITIONS_COL,
        NUM_OPEN_SHORT_POSITIONS_COL,
        NUM_OPEN_POSITIONS_COL,
    ],
)
fig.show()